In [1]:
%pip install transformers
%pip install sacremoses
%pip install torch
%pip install datasets



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import BioGptTokenizer, BioGptForCausalLM, Trainer, TrainingArguments
from datasets import Dataset
import torch

tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")
model = BioGptForCausalLM.from_pretrained("microsoft/biogpt")
model.to('cuda')

C:\Users\Almond\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Almond\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BioGptForCausalLM(
  (biogpt): BioGptModel(
    (embed_tokens): BioGptScaledWordEmbedding(42384, 1024, padding_idx=1)
    (embed_positions): BioGptLearnedPositionalEmbedding(1026, 1024)
    (layers): ModuleList(
      (0-23): 24 x BioGptDecoderLayer(
        (self_attn): BioGptSdpaAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (activation_fn): GELUActivation()
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      )
    )
    (layer_norm): LayerNorm((

In [3]:
import torch
torch.cuda.empty_cache()

In [14]:
from transformers import BioGptTokenizer, BioGptForCausalLM, Trainer, TrainingArguments
import pandas as pd
from datasets import Dataset

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=1024)

tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")
df = pd.read_csv('Question_1.csv') 

# Extract the abstracts
abstracts = df['Abstract']
abstracts = abstracts[:50]
print(abstracts)

dataset = Dataset.from_dict({"text": abstracts})
tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset.set_format("torch")
tokenized_dataset = tokenized_dataset.map(lambda x: {"labels": x["input_ids"]})

input_ids = tokenized_dataset['input_ids']
labels = input_ids.clone()

from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    lr_scheduler_type="linear",
    seed=42,
    per_device_train_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
    gradient_accumulation_steps=4,  # Accumulate gradients
    metric_for_best_model="loss",
    greater_is_better=False,
    load_best_model_at_end=True,
)

print(tokenized_dataset)

train_size = 30
val_size = 10
test_size = 10

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    tokenized_dataset, [train_size, val_size, test_size]
)

# Update Trainer with train and validation sets
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,    # Training set
    eval_dataset=val_dataset,       # Validation set for tuning during training
    data_collator=data_collator,
)

# Train the model
trainer.train()
metrics = trainer.evaluate(test_dataset)
print("Test set evaluation:", metrics)


C:\Users\Almond\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


50
                                             Abstract
0   Osimertinib is used as the first-line treatmen...
1   We wanted to present a rare case of metastatic...
2   Chemotherapy in an integral part of cancer tre...
3   DNA methylation plays a regulatory role in the...
4   Lung cancer is the leading cause of cancer-rel...
5   Diffuse large B-cell lymphoma is the most comm...
6   To evaluate the efficacy and safety of program...
7   Breast cancer (BC) is the most frequently occu...
8   Breast cancer has the potential to metastasize...
9   Bronchiolitis obliterans syndrome (BOS) occurr...
10  For drugs with a narrow therapeutic window, th...
11  The IROC head and neck phantom is used to cred...
12                             The discovery of  KRAS
13  To compare the efficacy of first-line regimens...
14  This systematic review and network meta-analys...
15  Accurate staging improves lung cancer survival...
16  This research aimed to systematically uncover ...
17  Lymph node metastasis

Map: 100%|██████████| 1/1 [00:00<00:00, 240.47 examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1
})



C:\Users\Almond\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ValueError: Sum of input lengths does not equal the length of the input dataset!

In [17]:
print(test_dataset)

prompt = "Treatment for lung cancer includes"
inputs = tokenizer(prompt, return_tensors="pt")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  
inputs = {key: val.to(device) for key, val in inputs.items()}

outputs = model.generate(
    input_ids=inputs["input_ids"],
    max_length=32,
    num_return_sequences=5, 
    no_repeat_ngram_size=2, 
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
)

for i in range(5):
    generated_text = tokenizer.decode(outputs[i], skip_special_tokens=True)
    print(generated_text)


Treatment for lung cancer includes surgery, radiation therapy, and chemotherapy. The role of chemotherapy in non-small cell lung carcinoma. Lung Cancer and Cancer Treatment.
Treatment for lung cancer includes surgical resection, radiation therapy, and chemotherapy. 1. Non-small-cell lung cancers.
Treatment for lung cancer includes surgery, radiation therapy, and chemotherapy.
Treatment for lung cancer includes surgical resection, chemotherapy, radiotherapy, and immunotherapy. The last decade has seen an increase in the utilization of minimally invasive lung resection.
Treatment for lung cancer includes the use of surgery, chemotherapy, radiation therapy, and immunotherapy.
